In [1]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )

testname="СКО"
display(Markdown('### 💠 {}'.format(testname)))

### 💠 Тест неравномерности АЧХ

In [2]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

FindDevice(children=(Dropdown(description='Выберите устройство:', layout=Layout(width='max-content'), options=…

In [3]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
result_config =  box.config

In [ ]:
def parse_config(config: dict):

    new_config = config.copy()
    
    if_value = config['DEVICE']['filter_if']
    at_value = config['DEVICE']['athenuator_value'] 
    
    
    new_config['DEVICE']['if_value']=if_value
    new_config['DEVICE']['athenuator_value']=at_value
    return new_config

In [ ]:
new_config=parse_config(dict(result_config))

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 21 09:28:15 2021

@author: kuzma.m
"""

import pyvisa as visa

#функция декоратор
def command_wrapper(f):
    def wrapped(inst, command):
        # print('До функции')
        try:
            if key == "debug":
                print(command)
        except:
            pass
        response = f(inst, command)
        # print('После функции')
        error = inst.query('SYST:ERR?')
        if error != '0, No error':
            print(error)
        return response

    # print('декорируем', f)
    return wrapped

#запрос данных
@command_wrapper
def query(inst, command):
    result = inst.query(command)
    return result

@command_wrapper
def query_ascii_values(inst, command):
    result = inst.query_ascii_values(command)
    return result

#запрос данных
@command_wrapper
def write(inst, command):
    inst.write(command)

#Функция конвертации множителей kGz,MGh,GHz в герцы
def HzConvertor(n,k):
    if k == "kHz":
        return int(n*1e+3)
    elif k == "MHz":
        return int(n*1e+6)
    elif k == "GHz":
        return int(n*1e+9)
    else: return -1

#Функция однократного росчерка	
def singlescan(connect):
    write(connect, f'TRIG:SOUR BUS')
    write(connect, f'INIT:CONT OFF')
    write(connect, f'INIT')
    write(connect, f'TRIG:SING')
    query(connect, f'*OPC?')

def SNVNAopen():
    # Подключение к SNVNA
    rm = visa.ResourceManager('@py')
    #Connect to a Socket on the local machine at 5025
    #Use the IP address of a remote machine to connect to it instead
    try:
        CMT = rm.open_resource('TCPIP0::localhost::5025::SOCKET')
    except:
        print("Failure to connect to VNA!")
        print("Check network settings")

    #The VNA ends each line with this. Reads will time out without this
    CMT.read_termination='\n'
    #Set a really long timeout period for slow sweeps
    CMT.timeout = 100000
    ##############################################
    return CMT

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 21 09:28:15 2021

@author: kuzma.m
"""

import pyvisa
import time
import json
#from connect import SNVNAopen, write, query, query_ascii_values, HzConvertor, singlescan




#кол-во портов
testname="Введите кол-во портов"
display(Markdown('### 💠 {}'.format(testname)))
c = int(input())

#алгоритм вычеслений СКО
def algoritm(attenuator,ifbw):

    #Установить фильтр ПЧ 300 кГц
    write(CMT, f'SENS:BWID {ifbw}')
    time.sleep(1) 

    #Частотный план 
    write(CMT, f'SERV:SWE:FREQ:FACT')

    #Attenuator control – вкл. 
    write(CMT, f'SERV:RFCTL:POW:STAT 1')
    write(CMT, f'SERV:RFCTL:POW:ATT {attenuator}')	

    #установить Код ЦАП.  АЧХ
    write(CMT, f'SERV:RFCTL:POW:DAC 6554')

    #росчерк трасс
    singlescan(CMT)

    tracc_count =  c
    #Создаем ветку словоря с шаблона 

    atthenuator["atthenuator"][f'{attenuator}']=atthenuator["atthenuator"]['DB']
    atthenuator["atthenuator"][f'{attenuator}']["IF"][f'{int(ifbw/1000)}'] = atthenuator["atthenuator"]['DB']["IF"]['GZ']
    #переменные для вывода максимумов
    max_sdev_abs = float(0)
    max_sdev_otn = float(0)
    #делаем нормализацию для каждой трассы
    for i in range(1,tracc_count+1):

        write(CMT, f'CALC1:PAR:COUN 3')
        write(CMT, f'CALC1:PAR1:DEF S{i}{i}')
        write(CMT, f'CALC1:PAR2:DEF R{i}')
        write(CMT, f'CALC1:PAR2:SPOR {i}')
        write(CMT, f'CALC1:PAR3:DEF T{i}')
        write(CMT, f'CALC1:PAR3:SPOR {i}')
        singlescan(CMT)
        for k in range(1,4):
            write(CMT,f'CALC:PAR{k}:SEL')
            write(CMT,f'CALC:MATH:MEM')
            write(CMT,f'CALC:MATH:FUNC DIV')
            #time.sleep(1)
        #росчерк трасс
        singlescan(CMT)

        for k in range(1,4):
            write(CMT,f'CALC:PAR{k}:SEL')
            write(CMT,f'CALC1:MST ON')
            track_def =  query(CMT,f'CALC1:PAR{k}:DEF?')
            #присвоение данных статистики
            mst_data = query_ascii_values(CMT,f'CALC:MST:DATA?')

            # поиск макс в S и ввод данных в словарь
            if k == 1:
                atthenuator["atthenuator"][f'{attenuator}']["IF"][f'{int(ifbw/1000)}']["trace"]["otnosit"][f'{track_def}'] = mst_data[1]
                if max_sdev_otn < float(mst_data[1]) or max_sdev_otn == 0:
                    max_sdev_otn = float(mst_data[1])
            # поиск макс в R и T и ввод данных в словарь
            else:
                atthenuator["atthenuator"][f'{attenuator}']["IF"][f'{int(ifbw/1000)}']["trace"]["absolute"][f'{track_def}'] = mst_data[1]
                if max_sdev_abs < float(mst_data[1]) or max_sdev_abs == 0:
                    max_sdev_abs = float(mst_data[1])

    #присвоение максимумов
    atthenuator["atthenuator"][f'{attenuator}']["IF"][f'{int(ifbw/1000)}']["trace"]["absolute"]['max_value'] = max_sdev_abs
    atthenuator["atthenuator"][f'{attenuator}']["IF"][f'{int(ifbw/1000)}']["trace"]["otnosit"]['max_value'] = max_sdev_otn

#Словарь для вывода в json файл(шаблон)
atthenuator = {"atthenuator": {
      "DB":{
        "IF":{
          "GZ":{
            "trace":{
              "absolute":{
                "max_value":"value"
              },
              "otnosit":{             
                "max_value":"value"
              }                         
            }
          },
        }
    }
}}

#Подключаемся к SNVNA
CMT = inst

#Пресет
write(CMT, f'SYST:PRES')

#Проверка кол-ва портов
if int(query(CMT,f'SERV:PORT:COUN?')) < c:
    print(f'Кол-во портов превышано\nЕсли хотите больше - смените прибор, в данный момент выставлено максимальное кол-во портов: {query(CMT,"SERV:PORT:COUN?")}')
    c = int(query(CMT,f'SERV:PORT:COUN?'))


""" выполнение (; 0_0 0_Q ZVO"""
# список проверяемых аттенюаторов и фильтров ПЧ 
     
Gz = new_config['DEVICE']['filter_if'].split(', ') 
DB = new_config['DEVICE']['athenuator_value'].split(', ') 

#выполнение функций
for i in DB:
    at_value = int(i)
    for j in Gz:
        if_value = int(j)
        algoritm(at_value,HzConvertor(if_value,"kHz"))
#удаление шаблона
for keys in atthenuator["atthenuator"].keys():
    if 'GZ' in atthenuator["atthenuator"][f'{keys}']["IF"].keys():
        del atthenuator["atthenuator"][f'{keys}']["IF"]['GZ']
del atthenuator["atthenuator"]['DB']

#запись в json
with open("sko_result.json", "w") as write_file:
    json.dump(atthenuator, write_file, indent=2)

In [ ]:
import json
from IPython.display import HTML, display

#создаем словарь для записи с json
data = {}

#открываем файл и загружаем данные
with open('sko_result.json', 'r', encoding='utf-8') as fh: 
    data.update(json.load(fh))

#ввод допуска
testname="Введите значение допуска"
display(Markdown('### 💠 {}'.format(testname)))
dop = float(input())

#создаем строку для ввода в html страницу
htmlstr = "<table border='1' ><tr><td>Канал</td><td>ПЧ, кГц</td>\
<td>Аттенюатор, дБ</td><td>Измерение, дБ</td><td>Максимум, дБ</td>\
<td>Допуск, дБ</td></tr><tr>\
"


#Цикл для заолнения html страницы
for i in data["atthenuator"].keys():
    for k in data["atthenuator"][f'{i}']["IF"].keys():
        for j in data["atthenuator"][f'{i}']["IF"][f'{k}']["trace"].keys():
            count = 0
            for n in data["atthenuator"][f'{i}']["IF"][f'{k}']["trace"][f'{j}'].keys():
                if n != "max_value":
                    count += 1
                    if count == 1:
                        htmlstr += f'<tr><td>{n}</td>'
                        l = data["atthenuator"][f'{i}']["IF"][f'{k}']["trace"][f'{j}'].keys()
                        htmlstr += f'<td rowspan={len(l)-1}>[{k}]</td><td rowspan={len(l)-1}>[{i}]</td>'
                        max_value = data["atthenuator"][f'{i}']["IF"][f'{k}']["trace"][f'{j}']["max_value"]
                        a = data["atthenuator"][f'{i}']["IF"][f'{k}']["trace"][f'{j}'][f'{n}']
                        if a > dop:
                            htmlstr += f'<td class = bg-warning>[{a}]</td><td rowspan={len(l)-1} >[{max_value}]</td>'
                        else:
                            htmlstr += f'<td>[{a}]</td><td rowspan={len(l)-1} >[{max_value}]</td>'

                        htmlstr += f'<td rowspan={len(l)-1}  >[{dop}]</td>'


                    else:
                        a = data["atthenuator"][f'{i}']["IF"][f'{k}']["trace"][f'{j}'][f'{n}']
                        if a > dop:
                            htmlstr += f'<tr><td>{n}</td><td class = bg-warning>[{a}]</td>'
                        else:
                            htmlstr += f'<tr><td>{n}</td><td >[{a}]</td>'



#записываем в файл
display(HTML(htmlstr))
with open('html_table.html', 'w', encoding='utf-8') as fh1: 
     fh1.write(htmlstr)

